In [31]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils import get_column_letter
import re
import xlsxwriter

In [33]:
portfolio_df = pd.read_csv('Robinhood-12-1-23.csv', on_bad_lines='skip')
portfolio_df = portfolio_df.drop(columns=['Process Date', 'Settle Date'])
portfolio_df = portfolio_df.rename(columns={'Instrument': 'Ticker'})
# Clean the 'Amount' column by removing '$' and converting '(value)' to negative values

portfolio_df['Amount'] = (
    portfolio_df['Amount']
    .replace('[\$,]', '', regex=True)       # Remove $ sign
    .replace('\((.*?)\)', r'-\1', regex=True)  # Convert (value) to -value
    .astype(float)                         # Convert the column to float type
)

description_splits = portfolio_df['Description'].str.split(' ', expand=True)

# The first part of the split is the 'Ticker', which we already have, so we ignore it
# The next parts are 'Expiry', 'Option Type', and 'Strike Price'
portfolio_df['Expiry'] = description_splits[1]
portfolio_df['Option Type'] = description_splits[2]
portfolio_df['Strike Price'] = description_splits[3]
portfolio_df = portfolio_df.drop(columns=['Description'])
# Define the new order of columns
new_column_order = [
    'Activity Date', 'Ticker', 'Expiry', 'Option Type', 'Strike Price',
    'Trans Code', 'Quantity', 'Price', 'Amount'
]

# Reorder the DataFrame according to the new column order
portfolio_df = portfolio_df[new_column_order]

portfolio_df.head()

,Activity Date,Ticker,Expiry,Option Type,Strike Price,Trans Code,Quantity,Price,Amount
0,12/1/2023,AAL,12/22/2023,Put,$11.50,BTO,1,$0.06,-6.03
1,12/1/2023,PFE,1/19/2024,Call,$33.00,BTO,1,$0.12,-12.03
2,12/1/2023,PFE,1/19/2024,Call,$34.00,STC,1,$0.06,5.95
3,12/1/2023,PFE,12/15/2023,Put,$28.50,STC,1,$0.42,41.95
4,12/1/2023,PFE,12/15/2023,Put,$26.50,BTC,1,$0.06,-6.03


In [34]:
output_excel_path = 'Modified_Robinhood_Report3.xlsx'
writer = pd.ExcelWriter(output_excel_path, engine='xlsxwriter')

# Write each ticker to a separate sheet
unique_tickers = portfolio_df['Ticker'].dropna().unique()
for ticker in unique_tickers:
    # Filter the dataframe for the ticker
    ticker_df = portfolio_df[portfolio_df['Ticker'] == ticker]
    # Write the dataframe to a separate sheet named after the ticker
    ticker_df.to_excel(writer, sheet_name=str(ticker), index=False)
    # Convert the data to an Excel table
    worksheet = writer.sheets[str(ticker)]  # Get the worksheet object
    (max_row, max_col) = ticker_df.shape
    # Create a table with the same size as the DataFrame
    column_settings = [{'header': column} for column in ticker_df.columns]
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

# Create a dashboard sheet - for now, it will just have a placeholder text
dashboard_df = pd.DataFrame({'Info': ['Dashboard placeholder - to be replaced with actual metrics']})
dashboard_df.to_excel(writer, sheet_name='Dashboard', index=False)

# Close the Pandas Excel writer and output the Excel file
writer.close()
output_excel_path

'Modified_Robinhood_Report3.xlsx'